In [2]:
import pandas as pd
import numpy as np

In [3]:
pop_2000 = pd.read_csv(
    "../00_source/sub-est00int.csv",
    encoding="latin-1",
    usecols=[
        "SUMLEV",
        "NAME",
        "STNAME",
        "POPESTIMATE2002",
        "POPESTIMATE2003",
        "POPESTIMATE2004",
        "POPESTIMATE2005",
        "POPESTIMATE2006",
        "POPESTIMATE2007",
        "POPESTIMATE2008",
        "POPESTIMATE2009",
    ],
)
pop_2010 = pd.read_csv(
    "../00_source/SUB-EST2020_ALL.csv",
    encoding="latin-1",
    usecols=[
        "SUMLEV",
        "NAME",
        "STNAME",
        "POPESTIMATE2010",
        "POPESTIMATE2011",
        "POPESTIMATE2012",
        "POPESTIMATE2013",
        "POPESTIMATE2014",
        "POPESTIMATE2015",
        "POPESTIMATE2016",
        "POPESTIMATE2017",
        "POPESTIMATE2018",
        "POPESTIMATE2019",
    ],
)


In [4]:
pop_2000 = pop_2000[pop_2000['SUMLEV'] == 162]
pop_2010 = pop_2010[pop_2010['SUMLEV'] == 162]

In [5]:
pop_2010["keep"] = False
pop_2000["keep"] = False
city_states = {
    "Atlanta city": "Georgia",
    "Boston city": "Massachusetts",
    "Chicago city": "Illinois",
    "Los Angeles city": "California",
    "New York city": "New York",
    "Philadelphia city": "Pennsylvania",
    "Portland city": "Oregon",
    "San Francisco city": "California",
    "Washington city": "District of Columbia",
}
for key, val in city_states.items():
    pop_2000.loc[
        (pop_2000.loc[:, "NAME"] == key) & (pop_2000.loc[:, "STNAME"] == val), "keep"
    ] = True
    pop_2010.loc[
        (pop_2010.loc[:, "NAME"] == key) & (pop_2010.loc[:, "STNAME"] == val), "keep"
    ] = True
    pass


In [6]:
pop_2000 = pop_2000[pop_2000["keep"] == True]
pop_2010 = pop_2010[pop_2010["keep"] == True]


In [7]:
pop = pop_2000.merge(pop_2010, how = "outer", on = ["NAME", "STNAME", "SUMLEV", "keep"], indicator = True, validate="1:1")

In [8]:
pop['City'] = pop['NAME'].str.strip(' city')
pop.loc[pop.loc[:,'City'] == 'Washington','City'] = 'Washington, D.C.'
pop.loc[pop.loc[:,'City'] == 'New York','City'] = 'New York City'

In [9]:
pop.drop(['SUMLEV', 'keep', '_merge', 'NAME', 'STNAME'], axis=1, inplace = True)

In [10]:
pop = pop.melt(id_vars=["City"], value_name="Population", var_name="Year")
pop['Year'] = pop['Year'].str.strip('POPESTIMATE').astype(int)

In [11]:
assert (pop['Population'] > 0).all()
assert ((pop['Year'] >= 2002) & (pop['Year'] <= 2019)).all()
assert (pop['City'].value_counts() == 18).all()

In [12]:
land_area = pd.read_excel("../00_source/land_area.xlsx")


In [13]:
pop = pop.merge(land_area, how = 'left', on = 'City')
pop['Population Density'] = pop["Population"]/pop["Land area"]
pop.drop(["Land area"], axis = 1, inplace = True)

In [14]:
unemployment = pd.read_csv('../00_source/unemployment.csv')

In [15]:
unemployment["id"] = unemployment["Series ID"].str[-1:].astype(int)
unemployment["City"] = unemployment["Series ID"].str[:-2].str.strip("0000000000000")
unemployment["id"] = np.where(
    unemployment["id"] == 3, "Unemployment Rate", "Labor Force"
)
mapping = {
    "LAUCT0644": "Los Angeles",
    "LAUCN06075": "San Francisco",
    "LAUCT115": "Washington, D.C.",
    "LAUCT1304": "Atlanta",
    "LAUCT1714": "Chicago",
    "LAUCT2507": "Boston",
    "LASCT3651": "New York City",
    "LAUCT4159": "Portland",
    "LAUCT426": "Philadelphia",
}
unemployment["City"] = unemployment["City"].map(mapping)


In [16]:
months = {
    "Jan": "January",
    "Feb": "February",
    "Mar": "March",
    "Apr": "April",
    "May": "May",
    "Jun": "June",
    "Jul": "July",
    "Aug": "August",
    "Sep": "September",
    "Oct": "October",
    "Nov": "November",
    "Dec": "December",
}
unemployment['Month'] = unemployment['Label'].str[-3:].map(months)
unemployment['Month Numeric'] = unemployment['Period'].str.strip("MO").astype(int)
unemployment.drop(["Series ID", "Period", "Label"], axis = 1, inplace = True)

In [17]:
unemployment = unemployment.pivot_table(
    values="Value", index=["City", "Year", "Month", "Month Numeric"], columns=["id"]
).reset_index()

In [18]:
city_stats = pop.merge(unemployment, how = "outer", on = ["City", "Year"], validate="1:m")

In [19]:
gas = pd.read_csv("../00_source/gas.csv")

In [20]:
gas["Month Numeric"] = gas["Period"].str.strip("MO").astype(int)
gas["Month"] = gas["Label"].str[-3:].map(months)
gas["Gas Price"] = gas["Value"]
gas.drop(["Series ID", "Period", "Label", "Value"], axis=1, inplace=True)


In [21]:
city_stats = city_stats.merge(
    gas,
    how="outer",
    on=["Year", "Month", "Month Numeric"],
    validate="m:1",
)


In [22]:
cpi = pd.read_csv("../00_source/cpi.csv")

In [23]:
cpi['Year'] = pd.DatetimeIndex(cpi["Date"]).year
cpi['Month Numeric'] = pd.DatetimeIndex(cpi["Date"]).month
cpi['CPI'] = cpi['CPI']/177.7
cpi.drop(["Date"], axis = 1, inplace=True)


In [24]:
city_stats = city_stats.merge(cpi, how = "outer", on = ["Year", "Month Numeric"], validate = "m:1")

In [25]:
city_stats.to_parquet(
    "../20_intermediate_data/city_statistics.parquet", engine="fastparquet"
)
